In [111]:
import json
import torch.nn as nn
import numpy as np
import h5py
import math
from tqdm import tqdm

## Load COOT Models

In [2]:
import torch
from nntrainer.models.transformer_legacy import TransformerLegacy
from nntrainer.models.transformer_legacy import TransformerConfig

In [3]:
transf_config = {'name': 'transformer', 'output_dim': 384, 'use_input_fc': True, 'input_fc_config': {'output_dim': 384, 'num_layers': 1, 'hidden_dim': 0, 'activation_middle': 'none', 'activation_output': 'gelu', 'dropout_middle': 0, 'dropout_output': 0, 'norm_middle': 'none', 'norm_output': 'none', 'residual': 'none'}, 'positional_encoding': 'sincos', 'add_local_cls_token': False, 'dropout_input': 0, 'norm_input': 'layernorm_coot', 'selfatn_config': {'hidden_dim': 384, 'num_layers': 1, 'num_heads': 8, 'pointwise_ff_dim': 384, 'activation': 'gelu', 'dropout': 0.05, 'norm': 'layernorm_coot'}, 'use_context': False, 'use_output_fc': False, 'pooler_config': {'name': 'atn', 'hidden_dim': 768, 'num_heads': 2, 'num_layers': 1, 'dropout': 0.05, 'activation': 'gelu'}, 'weight_init_type': 'truncnorm', 'weight_init_std': 0.01}
current_cfg = TransformerConfig(transf_config)
net_video_local = TransformerLegacy(current_cfg, 512)

In [4]:
net_video_local.load_state_dict(torch.load('provided_models/Net_local.pth'))
net_video_local.eval()

TransformerLegacy(
  (norm_input): LayerNormalization()
  (input_fc): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=512, out_features=384, bias=True)
    )
    (activation_output): GELU(approximate='none')
    (norm_output): Identity()
  )
  (embedding): PositionalEncodingSinCos(
    (dropout): Dropout(p=0, inplace=False)
  )
  (tf): TransformerEncoder(
    (encoder_layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attention_layer): Sublayer(
          (sublayer): MultiHeadAttention(
            (query_projection): Linear(in_features=384, out_features=384, bias=True)
            (key_projection): Linear(in_features=384, out_features=384, bias=True)
            (value_projection): Linear(in_features=384, out_features=384, bias=True)
            (final_projection): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.05, inplace=False)
            (softmax): Softmax(dim=3)
          )
          (layer_normalization): Lay

In [5]:
transf_config = {'name': 'transformer', 'output_dim': 768, 'use_input_fc': False, 'input_fc_config': None, 'use_context': True, 'crossatn_config': {'hidden_dim': 384, 'num_layers': 1, 'num_heads': 8, 'pointwise_ff_dim': 384, 'activation': 'gelu', 'dropout': 0.05, 'norm': 'layernorm_coot'}, 'pooler_config': {'name': 'avg_special'}, 'positional_encoding': 'sincos', 'add_local_cls_token': False, 'dropout_input': 0, 'norm_input': 'layernorm_coot', 'selfatn_config': {'hidden_dim': 384, 'num_layers': 1, 'num_heads': 8, 'pointwise_ff_dim': 384, 'activation': 'gelu', 'dropout': 0.05, 'norm': 'layernorm_coot'}, 'use_output_fc': False, 'weight_init_type': 'truncnorm', 'weight_init_std': 0.01}
current_cfg = TransformerConfig(transf_config)
coot = TransformerLegacy(current_cfg, 384)

In [6]:
coot.load_state_dict(torch.load('provided_models/TheModel.pth'))
coot.eval()

TransformerLegacy(
  (norm_input): LayerNormalization()
  (embedding): PositionalEncodingSinCos(
    (dropout): Dropout(p=0, inplace=False)
  )
  (tf): TransformerEncoder(
    (encoder_layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attention_layer): Sublayer(
          (sublayer): MultiHeadAttention(
            (query_projection): Linear(in_features=384, out_features=384, bias=True)
            (key_projection): Linear(in_features=384, out_features=384, bias=True)
            (value_projection): Linear(in_features=384, out_features=384, bias=True)
            (final_projection): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.05, inplace=False)
            (softmax): Softmax(dim=3)
          )
          (layer_normalization): LayerNormalization()
        )
        (pointwise_feedforward_layer): Sublayer(
          (sublayer): PointwiseFeedForwardNetwork(
            (feed_forward): Sequential(
              (0): Linear(in

### Summarizer Model

In [7]:
def knapsack_dp(values,weights,n_items,capacity,return_all=False):
    # check_inputs(values,weights,n_items,capacity)
    
    # assert(isinstance(values,list))
    # assert(isinstance(weights,list))
    # assert(isinstance(n_items,int))
    # assert(isinstance(capacity,int))
    # check value type
    # assert(all(isinstance(val,int) or isinstance(val,float) for val in values))
    # assert(all(isinstance(val,int) for val in weights))
    # check validity of value
    # assert(all(val >= 0 for val in weights))
    # assert(n_items > 0)
    # assert(capacity > 0)

    table = torch.zeros((n_items+1,capacity+1))
    keep = torch.zeros((n_items+1,capacity+1))

    for i in range(1,n_items+1):
        for w in range(0,capacity+1):
            wi = weights[i-1] # weight of current item
            vi = values[i-1] # value of current item
            if (wi <= w) and (vi + table[i-1,w-wi] > table[i-1,w]):
                table[i,w] = vi*1 + table[i-1,w-wi]
                keep[i,w] = 1
            else:
                table[i,w] = vi*0+table[i-1,w]
    
    
    picks = []
    cum_wghts = torch.zeros(n_items)
    K = capacity
    for i in range(n_items,0,-1):
        if keep[i,K] == 1:
            picks.append(i)
            cum_wghts[i-1] = table[i,K]
            K -= weights[i-1]

    picks.sort()
    picks = [x-1 for x in picks] # change to 0-index
    chosens = (cum_wghts-torch.min(cum_wghts)) / (torch.max(cum_wghts)-torch.min(cum_wghts))
    # print(picks, torch.ceil(chosens) )
    

    # if return_all:
    #     max_val = table[n_items,capacity]
    #     return picks,max_val
    return picks, torch.ceil(chosens)

In [8]:
class ShotScore(nn.Module):
    def __init__(self, input_size=384):
        super(ShotScore, self).__init__()
        self.Clip_Encoding = nn.Sequential(
                                nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True),
                                nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True))
        self.Clip_Scoring = nn.Sequential(
                                nn.Linear(in_features=input_size, out_features=input_size//2, bias=True),
                                nn.Linear(in_features=input_size//2, out_features=1, bias=True))
        
        self.softmax = nn.Softmax(dim=1)
        
    
    
    def forward(self,clip_embs, stages,  batch_mask=None):
        
        f1 = clip_embs
#         for s in range(stages):
#             clip_encoder = self.Clip_Encoding(Features)+Features
#             clip_Scores = self.Clip_Scoring(clip_encoder)

#     #         Scene_emb_list = []
#     #         for start, end in scene_boundaries:
#     #             scene_embs = self.scene_embs(clip_embs[start:end])
#     #             Scene_emb_list.append(scene_embs)

#     #             Scene_Scores = self.Scene_Scoring_Transformer(Scene_emb_list)

#     #         for start, end in scene_boundaries:  
#                 # clip_Scores+=Scene_Scores

#             si = self.softmax(clip_Scores)
#             progressive_scores.append(si)
#             Features += si*Features
        
        s1 = self.softmax(self.Clip_Scoring(self.Clip_Encoding(f1)+f1))
        
        f2 = s1*f1 + f1
        s2 = self.softmax(self.Clip_Scoring(self.Clip_Encoding(f2)+f2))
        
        f3 = s2*f2 + f2
        s3 = self.softmax(self.Clip_Scoring(self.Clip_Encoding(f3)+f3))
            
        
        scores = self.softmax(s1*s2*s3)
        return scores

### Trainer

In [9]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname == 'Linear':
        torch.nn.init.xavier_uniform_(m.weight, gain=np.sqrt(2.0))
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0.1)

In [50]:
def get_hidden_context(key, pred_clip_mask, nfps):
    with h5py.File('data/youcook2/video_feat_100m.h5', 'r') as d:
        vid_feats = torch.tensor(d[key][()])
        mask = torch.zeros(len(vid_feats))
        
        # start = 0
        # for i in range(len(nfps)):
        #     end = nfps[i]+start
        #     if i in picks:
        #         mask[start:end]=1
        #     start = end
        
        start = 0
        for i in range(len(nfps)):
            end = nfps[i]+start
            mask[start:end]=pred_clip_mask[i]
            start = end
            
        masked_vidfeats = vid_feats*mask.unsqueeze(0).T
        masked_vidfeats = masked_vidfeats.unsqueeze(0)
        masked_vidfeats = masked_vidfeats[masked_vidfeats.sum(dim=2) != 0].unsqueeze(0)
        nfs = mask.sum().int()
        mask = torch.zeros(1, nfs).bool()
        vid_context,_ = net_video_local(masked_vidfeats, mask, nfs, None)
        return vid_context
    
    

In [51]:
cos_sim = torch.nn.CosineSimilarity(dim=0, eps=0)
def diversity_loss(masked_vid):
    agg = []
    dl = 0
    for i in range(masked_vid.shape[0]):
        for j in range(i+1, masked_vid.shape[0]):
            yi = masked_vid[i]
            yj = masked_vid[j]
            cs = cos_sim(yi, yj)
            agg.append(cs)
            dl+=cs
            
    return torch.mean(torch.stack(agg))
        



In [155]:
def train(model, batch_size=1, device='cpu'):
    with open('frms_per_seg.json', 'r') as file:
        NFPS = json.load(file)
    l = nn.MSELoss()
    ScoreModel = model
    parameters = filter(lambda p: p.requires_grad, ScoreModel.parameters())
    optimizer = torch.optim.Adam(parameters, lr=0.00005, weight_decay=0.00001).to(device)
    prev_mask=0
    with h5py.File('experiments/retrieval/paper2020/yc2_100m_coot_valset1/embeddings/embeddings_0.h5', 'r') as d:
        clips = d['clip_num'][...]
        
        for epoch in range(50):
            pointer = 0
            epoch_loss = []
            
            print('Epoch:', epoch,'/50')
            pbar = tqdm(total=len(clips), position=0, leave=True)
            for clip_num, nclips in enumerate(clips):
                key = d['key'][clip_num].decode()
                # print(key)
                clip_embs = d['clip_emb'][pointer:pointer+nclips]
                ori_vid_cont = d['vid_context'][clip_num]
                ori_vid_emb = d['vid_emb'][clip_num]
                pointer += nclips

                # vid_cont = torch.tensor(vid_context).unsqueeze(0)
                clip_embs = torch.tensor(clip_embs).unsqueeze(0).to(device)
                ori_vid_cont = torch.tensor(ori_vid_cont).unsqueeze(0)
                ori_vid_emb = torch.tensor(ori_vid_emb).unsqueeze(0)


                scores = ScoreModel(clip_embs,3).reshape(-1)

                nfps = NFPS[key]

                picks, predicted_mask = knapsack_dp(scores.squeeze(0), NFPS[key], len(nfps), math.ceil(sum(nfps)*15/100))


                # print('###')
                # predicted_mask = torch.zeros(nclips)
                # predicted_mask[picks]=1

                # mask = torch.ceil(predicted_mask.unsqueeze(0)*scores)
                mask = predicted_mask.unsqueeze(0)


                # if clip_num==0:
                #     print(mask-prev_mask)
                #     prev_mask=mask
                    
                masked_vid = clip_embs*mask.T

                masked_vid = masked_vid[masked_vid.sum(dim=2) != 0].unsqueeze(0)

                ### Check masked_vid has grads
                clipn = predicted_mask.sum().int()
                mask = torch.zeros(1, clipn).bool()
                
                vid_cont = get_hidden_context(key, predicted_mask, nfps)

                
                vid_emb, _ = coot(masked_vid, mask, clipn, vid_cont)
                # print(ori_vid_emb.shape)
                # print(vid_emb.shape)

                div_loss = diversity_loss(masked_vid.squeeze())
                reconstruction_loss = l(vid_emb, ori_vid_emb)
                context_loss  = l(vid_cont, ori_vid_cont)
                Loss = (0.2*div_loss) + (0.5*reconstruction_loss) + (0.3*context_loss)

                # print("Loss:", Loss)
                optimizer.zero_grad()
                Loss.backward()
                optimizer.step()
                epoch_loss.append(Loss.item())
                
                pbar.update(1)
                pbar.set_description("Loss= %.3f" % Loss.item())
            pbar.close()
                
            print('Average Epoch Loss:', sum(epoch_loss)/len(epoch_loss))
            torch.save(ScoreModel.state_dict(), f'models/model@{epoch%5}.pth')
            
            if(epoch%5==0):
                fs_gt, fs_us = validation(ScoreModel)
                print('Avg GT FSore:',fs_gt, '   Avg User FSore:', fs_us)
            
            
    return(ScoreModel)
    
            

### Validation

In [144]:
def validation(ScoreModel):
    
    with open('frms_per_seg.json', 'r') as file:
        NFPS = json.load(file)
        
    avg_gtfs = []
    avg_usfs = []
    with h5py.File('experiments/retrieval/paper2020/yc2_100m_coot_valset1/embeddings/embeddings_0.h5', 'r') as d, h5py.File('summaries.h5','r') as summh5:
        clips = d['clip_num'][...]

        pointer = 0
        for clip_num, nclips in enumerate(clips):
            key = d['key'][clip_num].decode()
            clip_embs = d['clip_emb'][pointer:pointer+nclips]
            pointer += nclips

            clip_embs = torch.tensor(clip_embs).unsqueeze(0)

            scores = ScoreModel(clip_embs, 3).reshape(-1)

            nfps = NFPS[key]

            picks,_ = knapsack_dp(scores.squeeze(0), NFPS[key], len(nfps), math.ceil(sum(nfps)*15/100))
            
            machine_summary = get_machine_summ(picks, nfps)
            gt_summary = summh5[key]['gt_summary']
            user_summary = summh5[key]['user_summary']
            fs_gt,_,_ = evaluate_summary(machine_summary, np.expand_dims(gt_summary, 0) )
            fs_ut,_,_ = evaluate_summary(machine_summary, np.array(user_summary) )
            avg_gtfs.append(fs_gt)
            avg_usfs.append(avg_usfs)
            
        return fs_gt.mean(), fs_ut.mean()


In [133]:
def get_machine_summ(picks, nfps):
    tot_feats = sum(nfps)
    summ = []
    for n, nfs in enumerate(nfps):
        if n in picks:
            summ = np.append(summ, np.ones(16*nfs))
        else:
            summ = np.append(summ, np.zeros(16*nfs))
    return summ

In [136]:
def evaluate_summary(machine_summary, user_summary, eval_metric='avg'):
    """Compare machine summary with user summary (keyshot-based).
    Args:
    --------------------------------
    machine_summary and user_summary should be binary vectors of ndarray type.
    eval_metric = {'avg', 'max'}
    'avg' averages results of comparing multiple human summaries.
    'max' takes the maximum (best) out of multiple comparisons.
    """
    machine_summary = machine_summary.astype(np.float32)
    user_summary = user_summary.astype(np.float32)
    n_users,n_frames = user_summary.shape

    # binarization
    machine_summary[machine_summary > 0] = 1
    user_summary[user_summary > 0] = 1

    if len(machine_summary) > n_frames:
        machine_summary = machine_summary[:n_frames]
    elif len(machine_summary) < n_frames:
        zero_padding = np.zeros((n_frames - len(machine_summary)))
        machine_summary = np.concatenate([machine_summary, zero_padding])

    f_scores = []
    prec_arr = []
    rec_arr = []

    for user_idx in range(n_users):
        gt_summary = user_summary[user_idx,:]
        overlap_duration = (machine_summary * gt_summary).sum()
        precision = overlap_duration / (machine_summary.sum() + 1e-8)
        recall = overlap_duration / (gt_summary.sum() + 1e-8)
        if precision == 0 and recall == 0:
            f_score = 0.
        else:
            f_score = (2 * precision * recall) / (precision + recall)
        f_scores.append(f_score)
        prec_arr.append(precision)
        rec_arr.append(recall)

    final_f_score = np.mean(f_scores)
    final_prec = np.mean(prec_arr)
    final_rec = np.mean(rec_arr)
    return final_f_score, final_prec, final_rec

## Testing

In [157]:
model = ShotScore()
# model.load_state_dict(torch.load('model/model@i.pth', map_location=lambda storage, loc: storage))
model.apply(weights_init)
# model.to('gpu')
trained_model = train(model)

Epoch: 0 /50


Loss= 0.834:   7%|▋         | 5/72 [00:04<00:51,  1.31it/s]

KeyboardInterrupt: 

In [5]:
import torch
x=torch.tensor([2.2,3.6])
x.sum().int().unsqueeze(-1)

tensor([5], dtype=torch.int32)

In [7]:
torch.tensor(5).int()

tensor(5, dtype=torch.int32)